In [ ]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from transformers import PreTrainedTokenizerFast
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
ds = load_dataset("habanoz/news-tr-1.8M")

In [ ]:
shuffledData = ds['train'].shuffle(seed=42)
oneFifthData = shuffledData.select(range(len(ds['train']) // 5))

In [ ]:
llama32tokenizer = Tokenizer(BPE(unk_token="<unk>"))
trainer = BpeTrainer(special_tokens=["<unk>", "<pad>", "<bos>", "<eos>"], vocab_size=32768)

def get_training_corpus():
    for sample in oneFifthData:
        yield sample['text']

llama32tokenizer.train_from_iterator(get_training_corpus(), trainer)

In [ ]:
fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=llama32tokenizer,
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>"
)

In [ ]:
fast_tokenizer.push_to_hub("aliarda/turkish-news-32k-tokenizer")